In [4]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import time
import requests
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

In [5]:
public = pd.read_csv('../data/Public_Housing_Buildings.csv')
public.columns = [x.lower() for x in public.columns]

C:\Users\jackv\AppData\Local\Temp\ipykernel_17976\3849602824.py:1: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  public = pd.read_csv('../data/Public_Housing_Buildings.csv')


In [6]:
len(public)

191885

In [7]:
public.sample()

,objectid,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,national_bldg_id,construct_date,dofa_actual_dt,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_no,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,x,y
36895,36896,FL001,Jacksonville Housing Authority,FL001000013,SOUTHWIND VILLAS,8711 Newton Rd. Apt. 73-74,1317,SD,INAPCP,6.735400e+09,10/31/1972 12:00:00 AM,10/31/1972 12:00:00 AM,2,2,2,0,2,100.0,2,-4.0,-4,-4.0,-4.0,-4,-4,-4,-4,-4,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4,-4.0,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,-4.0,-4,12.0,Duval,31.0,15923.0,1.0,1009.0,NaN,NaN,Duval,31.0,91642.0,Jacksonville East CC,35000.0,Jacksonville city (part),C1,NaN,3600.0,"Jacksonville, FL MSA",27260.0,"Jacksonville, FL Metropolitan Statistical Area",99999.0,NaN,1.0,0.0,1205,1458.0,NaN,NaN,8711 Newton Rd,NaN,Jacksonville,FL,32216.0,410.0,NaN,32216.0,99.0,3.0,3.221604e+10,H,NaN,NaN,NaN,1.0,NaN,NaN,A,NaN,90% Overall probable correctness,30.275712,-81.561855,S,NaN,R,U,** Street-Level Rooftop **,12031.0,1235000.0,1.203102e+10,1.203102e+11,N,,3/6/2023 8:50:52 AM,-4,-4.0,2314,FL00100001313171,-9.079424e+06,3.539040e+06


In [8]:
dev = pd.read_csv('../data/Public_Housing_Developments.csv')
dev.columns = [x.lower() for x in dev.columns]
dev = dev[dev.project_name.isin(public.project_name)]

In [9]:
dev.sample()

,objectid,participant_code,formal_participant_name,development_code,project_name,scattered_site_ind,pd_status_type_code,total_units,total_dwelling_units,acc_units,total_occupied,regular_vacant,pha_total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_no,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,spending_per_month_prev_yr,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,x,y
3794,3795,GA064,Housing Authority of the City of Bainbridge,GA064000001,"HUTTO-MCIVER HOMES,Sims & Scatters Sites",N,M,141,139,139,124,12,283,89.21,124,100.0,5,31.45,2.7,335,216,693,10119,3746,33.87,16.13,29.03,5.65,15.32,38.71,0.81,60.48,18.51,94.35,76.61,0.81,75.0,90.32,73.39,16.24,57.14,8.06,18.55,62.9,12.9,5.65,0.0,91.13,90.32,0.0,0.0,0.81,2,40,100.0,37,16.13,40.32,43.55,5.65,61.17,35.77,63.15,13.0,Decatur,87.0,970400.0,3.0,3029.0,NaN,NaN,Decatur,87.0,90192.0,Bainbridge,4896.0,Bainbridge city (part),C1,NaN,9999.0,NaN,12460.0,"Bainbridge, GA Micropolitan Statistical Area",99999.0,NaN,0.0,1.0,1302,228.0,NaN,NaN,108 S Sims St,NaN,Bainbridge,GA,39817.0,4175.0,NaN,39819.0,99.0,7.0,3.981742e+10,H,1SUD,NaN,NaN,2.0,NaN,NaN,N,NaN,80% Overall probable correctness,30.905582,-84.561604,S,NaN,R,U,** Street-Level Rooftop **,13087.0,1304896.0,1.308797e+10,1.308797e+11,473,195,2.09,6.87,99.190002,10032,3/6/2023 7:35:49 AM,1030795,646761.0,-9.413355e+06,3.620493e+06


In [10]:
dev_to_add = dev.loc[:, [
    "development_code",
    "spending_per_month_prev_yr",
    "chldrn_mbr_cnt",
    "annl_expns_amnt",
    "annl_expns_amnt_prev_yr",
    "eldly_prcnt",
    "pct_disabled_lt62_all",
    "pct_lt80_median",
    "median_inc_amnt",
    "scattered_site_ind",
    "pd_status_type_code",
    "total_units",
    "total_dwelling_units",
    "acc_units",
    "total_occupied",
    "regular_vacant",
    "pha_total_units",
    "pct_occupied",
    "number_reported",
    "pct_reported",
    "months_since_report",
    "pct_movein",
    "people_per_unit",
    "people_total",
    "rent_per_month",
    "spending_per_month",
    "hh_income",
    "person_income",
    "pct_lt5k",
    "pct_5k_lt10k",
    "pct_10k_lt15k",
    "pct_15k_lt20k",
    "pct_ge20k",
    "pct_wage_major",
    "pct_welfare_major",
    "pct_other_major",
    "pct_median",
    "pct_lt50_median",
    "pct_lt30_median",
    "pct_2adults",
    "pct_1adult",
    "pct_female_head",
    "pct_female_head_child",
    "pct_disabled_lt62",
    "pct_disabled_ge62",
    "pct_disabled_all",
    "pct_lt24_head",
    "pct_age25_50",
    "pct_age51_61",
    "pct_age62plus",
    "pct_age85plus",
    "pct_minority",
    "pct_black",
    "pct_native_american",
    "pct_asian",
    "pct_hispanic",
    "months_waiting",
    "months_from_movein",
    "pct_utility_allow",
    "ave_util_allow",
    "pct_bed1",
    "pct_bed2",
    "pct_bed3",
    "pct_overhoused",
    "tminority",
    "tpoverty",
    "tpct_ownsfd"
]].copy()
dev_to_add.columns = ['dev_' + str(x) for x in dev_to_add.columns]

In [11]:
public.development_code = public.development_code.str.upper()
dev.dev_development_code = dev_to_add.dev_development_code.str.upper()

C:\Users\jackv\AppData\Local\Temp\ipykernel_17976\779776153.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dev.dev_development_code = dev_to_add.dev_development_code.str.upper()


In [12]:
public = public.merge(dev_to_add, left_on='development_code', 
             right_on='dev_development_code', how='left')   

In [13]:
len(public)

191885

In [14]:
pha = pd.read_csv('../data/Public_Housing_Authorities.csv')
pha.columns = [x.lower() for x in pha.columns]

In [15]:
public.sample()

,objectid,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,national_bldg_id,construct_date,dofa_actual_dt,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_no,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,x,y,dev_development_code,dev_spending_per_month_prev_yr,dev_chldrn_mbr_cnt,dev_annl_expns_amnt,dev_annl_expns_amnt_prev_yr,dev_eldly_prcnt,dev_pct_disabled_lt62_all,dev_pct_lt80_median,dev_median_inc_amnt,dev_scattered_site_ind,dev_pd_status_type_code,dev_total_units,dev_total_dwelling_units,dev_acc_units,dev_total_occupied,dev_regular_vacant,dev_pha_total_units,dev_pct_occupied,dev_number_reported,dev_pct_reported,dev_months_since_report,dev_pct_movein,dev_people_per_unit,dev_people_total,dev_rent_per_month,dev_spending_per_month,dev_hh_income,dev_person_income,dev_pct_lt5k,dev_pct_5k_lt10k,dev_pct_10k_lt15k,dev_pct_15k_lt20k,dev_pct_ge20k,dev_pct_wage_major,dev_pct_welfare_major,dev_pct_other_major,dev_pct_median,dev_pct_lt50_median,dev_pct_lt30_median,dev_pct_2adults,dev_pct_1adult,dev_pct_female_head,dev_pct_female_head_child,dev_pct_disabled_lt62,dev_pct_disabled_ge62,dev_pct_disabled_all,dev_pct_lt24_head,dev_pct_age25_50,dev_pct_age51_61,dev_pct_age62plus,dev_pct_age85plus,dev_pct_minority,dev_pct_black,dev_pct_native_american,dev_pct_asian,dev_pct_hispanic,dev_months_waiting,dev_months_from_movein,dev_pct_utility_allow,dev_ave_util_allow,dev_pct_bed1,dev_pct_bed2,dev_pct_bed3,dev_pct_overhoused,dev_tminority,dev_tpoverty,dev_tpct_ownsfd
21920,21921,SC005,HOUSING AUTHORITY OF DARLINGTON,SC005000001,PINE VILLAGE,BOWEN MANOR,057,WU,INAPCP,5.016120e+09,8/1/1980 12:00:00 AM,7/31/1981 12:00:00 AM,2,2,2,0,2,100.0,2,-4.0,-4,-4.0,-4.0,-4,-4,-4,-4,-4,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4,-4.0,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,-4.0,-4,45.0,Darlington,31.0,11400.0,1.0,1003.0,NaN,NaN,Darlington,31.0,90858.0,Darlington,18565.0,Darlington city,C1,NaN,9999.0,NaN,22500.0,"Florence, SC Metropolitan Statistical Area",99999.0,NaN,1.0,0.0,4507,420.0,NaN,NaN,157 Bowens St,NaN,Darlington,SC,29532.0,4161.0,NaN,29532.0,57.0,5.0,2.953242e+10,S,NaN,NaN,NaN,2.0,NaN,NaN,A,NaN,80% Overall probable correctness,34.295544,-79.854232,S,NaN,R,U,** Street-Level Rooftop **,45031.0,4518565.0,4.503101e+10,4.503101e+11,N,,3/6/2023 8:50:52 AM,-4,-4.0,167,SC00500000105701,-8.889332e+06,4.068556e+06,SC005000001,554.0,251.0,1272707.0,1103872.0,7.14,6.93,91.57,14009.0,N,M,168.0,167.0

In [16]:
pha_to_add = pha.loc[:, [
    'participant_code', 
    'phas_designation',
    'ha_low_rent_size_category',
    'section8_units_cnt',
    'ha_section_8_size_category',
    'ha_combined_size_category',
    'ha_fye',
    'ha_program_type',
    'section8_occupied',
    'opfund_amnt',
    'opfund_amnt_prev_yr',
    'capfund_amnt',
    'capfund_amnt_prev_yr',
    'ross_amnt',
    'fss_amnt',
    'chldrn_mbr_cnt',
    'eldly_prcnt',
    'pct_disabled_lt62_all',
    'pct_lt80_median',
    'median_inc_amnt',
    'annl_expns_amnt',
    'annl_expns_amnt_prev_yr'
]].copy()
pha_to_add.columns = ['pha_' + str(x) for x in pha_to_add.columns]

In [17]:
pha_to_add.pha_participant_code = pha_to_add.pha_participant_code.str.upper()
public.participant_code = public.participant_code.str.upper()

In [18]:
public = public.merge(pha_to_add, left_on='participant_code', 
             right_on='pha_participant_code', how='left')   

In [19]:
len(public)

191885

In [20]:
public['std_addr'] = public.std_addr.str.upper()
public['std_st'] = public.std_st.str.upper()
public['std_city'] = public.std_city.str.upper()

In [21]:
def correct_address(std_addr):
    if pd.isna(std_addr):
        return np.nan, np.nan
    if 'APT' in std_addr:
        split_addr = std_addr.split('APT')
        return split_addr[0].strip(), split_addr[1].strip()
    if 'UNIT' in std_addr:
        split_addr = std_addr.split('UNIT')
        return split_addr[0].strip(), split_addr[1].strip()
    if 'SUB' in std_addr:
        split_addr = std_addr.split('SUB')
        return split_addr[0].strip(), split_addr[1].strip()
    if '#' in std_addr:
        split_addr = std_addr.split('#')
        return split_addr[0].strip(), split_addr[1].strip()
    if 'LOT' in std_addr:
        split_addr = std_addr.split('LOT')
        return split_addr[0].strip(), split_addr[1].strip()
    return std_addr, np.nan

public[['corrected_address', 'corrected_unit']] = public.std_addr.apply(correct_address).apply(pd.Series)

In [22]:
public['corrected_address'] = public.corrected_address.str.replace('OFC', '')
public['corrected_address'] = public.corrected_address.str.replace('1SUD', '')

In [23]:
public.corrected_unit = public.corrected_unit.fillna('')

In [24]:
public = public.drop(['apt_no'], axis=1)

Process NYC city names

In [25]:
public[public.std_st == 'NY'].groupby('std_city').size().sort_values(ascending=False).head(15)

std_city
BROOKLYN         720
NEW YORK         583
BUFFALO          560
ROCHESTER        548
BRONX            518
QUEENS           206
SYRACUSE         176
NIAGARA FALLS    171
ALBANY           136
UTICA             85
STATEN ISLAND     78
SCHENECTADY       77
PLATTSBURGH       68
KINGSTON          59
ELMIRA            54
dtype: int64

In [26]:
public.std_city = public.std_city.str.replace('STATEN ISLAND', 'NEW YORK')
public.std_city = public.std_city.str.replace('QUEENS', 'NEW YORK')
public.std_city = public.std_city.str.replace('BRONX', 'NEW YORK')
public.std_city = public.std_city.str.replace('MANHATTAN', 'NEW YORK')
public.std_city = public.std_city.str.replace('BROOKLYN', 'NEW YORK')
public.std_city = public.std_city.str.replace('NEW YORK CITY', 'NEW YORK')

In [27]:
public[public.std_st == 'NY'].groupby('std_city').size().sort_values(ascending=False).head(15)

std_city
NEW YORK            2116
BUFFALO              560
ROCHESTER            548
SYRACUSE             176
NIAGARA FALLS        171
ALBANY               136
UTICA                 85
SCHENECTADY           77
PLATTSBURGH           68
KINGSTON              59
ELMIRA                54
LOCKPORT              46
WATERTOWN             45
SARATOGA SPRINGS      43
BINGHAMTON            42
dtype: int64

In [28]:
public.sample()

,objectid,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,national_bldg_id,construct_date,dofa_actual_dt,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,x,y,dev_development_code,dev_spending_per_month_prev_yr,dev_chldrn_mbr_cnt,dev_annl_expns_amnt,dev_annl_expns_amnt_prev_yr,dev_eldly_prcnt,dev_pct_disabled_lt62_all,dev_pct_lt80_median,dev_median_inc_amnt,dev_scattered_site_ind,dev_pd_status_type_code,dev_total_units,dev_total_dwelling_units,dev_acc_units,dev_total_occupied,dev_regular_vacant,dev_pha_total_units,dev_pct_occupied,dev_number_reported,dev_pct_reported,dev_months_since_report,dev_pct_movein,dev_people_per_unit,dev_people_total,dev_rent_per_month,dev_spending_per_month,dev_hh_income,dev_person_income,dev_pct_lt5k,dev_pct_5k_lt10k,dev_pct_10k_lt15k,dev_pct_15k_lt20k,dev_pct_ge20k,dev_pct_wage_major,dev_pct_welfare_major,dev_pct_other_major,dev_pct_median,dev_pct_lt50_median,dev_pct_lt30_median,dev_pct_2adults,dev_pct_1adult,dev_pct_female_head,dev_pct_female_head_child,dev_pct_disabled_lt62,dev_pct_disabled_ge62,dev_pct_disabled_all,dev_pct_lt24_head,dev_pct_age25_50,dev_pct_age51_61,dev_pct_age62plus,dev_pct_age85plus,dev_pct_minority,dev_pct_black,dev_pct_native_american,dev_pct_asian,dev_pct_hispanic,dev_months_waiting,dev_months_from_movein,dev_pct_utility_allow,dev_ave_util_allow,dev_pct_bed1,dev_pct_bed2,dev_pct_bed3,dev_pct_overhoused,dev_tminority,dev_tpoverty,dev_tpct_ownsfd,pha_participant_code,pha_phas_designation,pha_ha_low_rent_size_category,pha_section8_units_cnt,pha_ha_section_8_size_category,pha_ha_combined_size_category,pha_ha_fye,pha_ha_program_type,pha_section8_occupied,pha_opfund_amnt,pha_opfund_amnt_prev_yr,pha_capfund_amnt,pha_capfund_amnt_prev_yr,pha_ross_amnt,pha_fss_amnt,pha_chldrn_mbr_cnt,pha_eldly_prcnt,pha_pct_disabled_lt62_all,pha_pct_lt80_median,pha_median_inc_amnt,pha_annl_expns_amnt,pha_annl_expns_amnt_prev_yr,corrected_address,corrected_unit
143767,143768,VA001,Portsmouth Redevelopment & Housing Authority,VA001000017,Seaboard Square II,NaN,SBT206,RW,INAPCP,8.399030e+09,3/29/2012 12:00:00 AM,5/31/2012 12:00:00 AM,3,3,3,0,3,100.0,3,-4.0,-4,-4.0,-4.0,-4,-4,-4,-4,-4,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4,-4.0,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,-4.0,-4,51.0,Portsmouth,740.0,211400.0,2.0,2010.0,NaN,NaN,Portsmouth,740.0,95123.0,Portsmouth,64000.0,Portsmouth city,C7,NaN,

In [29]:
public.to_csv('../data/buildings_dev_pha_2.1.csv', index=False)